<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Amostra%20desbalanceada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Referências

https://www.kaggle.com/saurav9786/feature-engineering-up-and-down-sampling

https://towardsdatascience.com/dealing-with-imbalanced-data-in-churn-analysis-6ea1afba8b5e

https://www.kdnuggets.com/2019/05/fix-unbalanced-dataset.html

https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb

https://medium.com/analytics-vidhya/balance-your-data-using-smote-98e4d79fcddb

https://www.geeksforgeeks.org/ml-handling-imbalanced-data-with-smote-and-near-miss-algorithm-in-python/

## Reamostragem da classe minoritária
* Up-sampling é o processo de reamostrar observações aleatoriamente da classe minoritária.

#### Carregar as libraries:

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt # importing ploting libraries
import seaborn as sns # importing seaborn for statistical plots
from collections import Counter

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, recall_score # calculate accuracy measures and confusion matrix

from imblearn.over_sampling import SMOTE

In [ ]:
!pip install imbalanced-learn

In [ ]:
%matplotlib inline

plt.rcParams['figure.figsize'] = [20.0, 7.0]
plt.rcParams.update({'font.size': 22,})

sns.set_palette('viridis')
sns.set_style('white')
sns.set_context('talk', font_scale = 0.8)

#### Carregar os dados
* Dataframe Credit Card: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

In [ ]:
url = '/root/.cache/kagglehub/datasets/mlg-ulb/creditcardfraud/versions/3/creditcard.csv'
df_cc = pd.read_csv(url)

df_cc.columns = [colunas.lower() for colunas in df_cc.columns]

df_cc.drop(columns = 'time', axis = 1, inplace = True)
df_cc.head()

### Tratamento dos Missing Values

In [ ]:
df_cc.isna().sum()

### Qual a proporção de fraudes e não-fraudes?

In [ ]:
qtd = Counter(df_cc['class'])
qtd

In [ ]:
np.round(100*qtd[1]/qtd[0], 4)

### Normalização

In [ ]:
df_cc2 = df_cc.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

l_colunas = df_cc.columns
l_colunas = l_colunas.drop('class')
l_colunas

In [ ]:
for coluna in l_colunas:
    df_cc2[coluna+'_2'] = StandardScaler().fit_transform(np.array(df_cc2[coluna]).reshape(-1, 1))
    df_cc2 = df_cc2.drop(columns = coluna, axis = 1)

df_cc2.head()

In [ ]:
X_cc = df_cc2.copy()
X_cc = X_cc.drop(columns = 'class', axis = 1)

y_cc = df_cc2['class']

### Amostra de treinamento e validação

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X_cc, y_cc, test_size = 0.3, random_state = 20111974)

### Treinamento do modelo

In [ ]:
# Instancia:
lr = LogisticRegression()

In [ ]:
# Treina o modelo usando a amostra de treinamento:
lr.fit(X_treinamento, y_treinamento.ravel())

### Previsão do modelo

In [ ]:
y_pred = lr.predict(X_teste)

# print classification report
print(classification_report(y_teste, y_pred))

**Conclusão**: Temos acurácia de 100%.

Observe o recall da classe minoritária: 0.57 ==> Isso mostra que estamos diante de uma amostra desbalanceada e que, neste caso, o modelo está inclinado/viesado pela classe majoritária.

## Reamostragem da classe Majoritária
* Up-sampling é o processo de reamostrar observações aleatoriamente da classe minoritária.

In [ ]:
X_cc = df_cc2.copy()
X_cc = X_cc.drop(columns = 'class', axis = 1)

y_cc = df_cc2['class']

### Processo
1. Separar as observações de cada classe em diferentes dataframes;
2. Reamostrar a classe minoritária COM REPOSIÇÃO;
3. Combinar os dois dataframes com as classes minoritárias e majoritárias.

Abaixo, seleção das instâncias/linhas em que [class] = 0 ==> Classe Majoritária

In [ ]:
df_cc_majo = df_cc2[df_cc2['class'] == 0]
df_cc_majo.shape

Abaixo, seleção das instâncias/linhas em que [class] = 1 ==> Classe Minoritária

In [ ]:
df_cc_mino = df_cc2[df_cc2['class'] == 1]
df_cc_mino.shape

## Reamostragem da classe majoritária (COM REPOSIÇÃO)

In [ ]:
np.random.seed(20111974)
df_cc_majo_s = df_cc_majo.sample(n = df_cc_mino.shape[0]+300, replace = True)
df_cc_majo_s.shape

#### Combinar os dois dataframes

In [ ]:
df_cc_s1 = pd.concat([df_cc_majo_s, df_cc_mino])
Counter(df_cc_s1['class'])

In [ ]:
df_cc_s1.head()

Portanto, o dataframe df_cc_s1 é uma das amostras em que tratamos o desbalanceamento.

### Amostra de treinamento e validação

In [ ]:
X = df_cc_s1.copy()
X = X.drop(columns = 'class', axis = 1)

y = df_cc_s1['class']

In [ ]:
from sklearn.model_selection import train_test_split

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3, random_state = 20111974)

### Treinamento do modelo

In [ ]:
# Instancia
from sklearn.linear_model import  LogisticRegression

lr = LogisticRegression()

In [ ]:
# treina o modelo na amostra de treinamento
lr.fit(X_treinamento, y_treinamento.ravel())

### Previsão do modelo

In [ ]:
y_pred = lr.predict(X_teste)

# print classification report
print(classification_report(y_teste, y_pred))

**Conclusão**: Temos acurácia de 94%.

Observe o recall da classe minoritária: 0.92 ==> Isso mostra que estamos diante de uma amostra balanceada.

### Verificar a quantidade de instâncias por preditoras

In [ ]:
X.shape[0]/X.shape[1]

Temos 44 linhas para cada coluna/variável do dataframe.

## SMOTE (Synthetic Minority Oversampling Technique)
* Uma forma de se resolver o problema das amostras desbalanceadas é simplesmente reamostrando a classe minoritária e isso pode ser obtido através da duplicação da classe minoritária. Isso resolve o problema do desbalanceamento, mas não traz nenhuma informação adicional ao modelo.
* Uma alternativa é criar amostras sintéticas da classe minoritária e pode ser efetivo para resolver o problema do desbalanceamento.
* A estratégia mais utilizada é o SMOTE.
    * Seleciona aleatoriamente amostras que estão próximos (amostras sintéticas).

SMOTE first selects a minority class instance a at random and finds its k nearest minority class neighbors. The synthetic instance is then created by choosing one of the k nearest neighbors b at random and connecting a and b to form a line segment in the feature space. The synthetic instances are generated as a convex combination of the two chosen instances a and b.

**Imbalanced Learning: Foundations, Algorithms, and Applications, 2013**


